In [1]:
from util import *

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import dtale
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [3]:
url = "https://www.datafiles.samhsa.gov/sites/default/files/field-uploads-protected/studies/TEDS-A-2018/TEDS-A-2018-datasets/TEDS-A-2018-DS0001/TEDS-A-2018-DS0001-bundles-with-study-info/TEDS-A-2018-DS0001-bndl-data-csv_v1.zip"
# df = read_csv(url) # Use this if not inside Coursera...
df = read_csv('TEDS-D-2019-DS0001-bndl-data-tsv_V1.zip')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.nunique(axis=0)

## PCA Testing (Select Features, 2 components) Uncomment to view charts

In [4]:
test_df = df.head()

features = ['LOS','SERVICES','PSOURCE','SUB1','ROUTE1','FREQ1','DSMCRIT','PSYPROB','HLTHINS']
test_df = preprocess_df(test_df, features, StandardScaler())

pca, transformed_X = unsupervised_model(test_df, features, PCA(n_components=2))

principalDf = pd.DataFrame(data = transformed_X, columns = ['PC1','PC2'])

final_df = pd.concat([principalDf, test_df[['REASON']]], axis=1)

# pca_scatter_plot(final_df, 'PC1', 'PC2', 'REASON')

## PCA Testing (All Columns, 2 components) Uncomment to view charts

In [ ]:
#All Columns
test_df = df.head()

features = df.columns.drop(['REASON'])
test_df = preprocess_df(test_df, features, StandardScaler())

pca, transformed_X = unsupervised_model(test_df, features, PCA(n_components=2))

principalDf = pd.DataFrame(data = transformed_X, columns = ['PC1','PC2'])

final_df = pd.concat([principalDf, test_df[['REASON']]], axis=1)

# pca_scatter_plot(final_df, 'PC1', 'PC2', 'REASON')

### PCA Variance and Clustering? Uncomment to view charts

In [ ]:
test_df = df.head(n=20)

pca, transformed_X = unsupervised_model(test_df, features, PCA(n_components=20))

components = range(pca.n_components_)

# bar_chart(components, pca.explained_variance_ratio_, 'PCA features', 'variance %')

Looks like a slight dropoff after four components.

In [ ]:
PCA_components_df = pd.DataFrame(transformed_X)

# %matplotlib inline
# plt.scatter(PCA_components_df[0], PCA_components_df[1], alpha=.01, color='black') #Alpha reduced to look for clustering
# plt.xlabel('PCA 1')
# plt.ylabel('PCA 2')
# plt.show()

Roughly three clusters? Hard to reduce alpha much further for clarity.

In [ ]:
ks = range(1, 10)
inertias = []
for k in ks:
    # Create a KMeans instance with k clusters: model
    model = KMeans(n_clusters=k)
    
    # Fit model to samples
    model.fit(PCA_components_df.iloc[:,:3])
    
    # Append the inertia to the list of inertias
    inertias.append(model.inertia_)
    
plt.plot(ks, inertias, '-o', color='black')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

Slight elbow after 3 clusters.

## Completion Rates by...

### State

In [ ]:
df2 = df.copy()
state_completion_df = df2.groupby(['STFIPS','REASON']).size().reset_index()

In [ ]:
state_completion_df = state_completion_df.rename(columns={0:'COUNT'})

In [ ]:
state_completion_df['PERC'] = 100 * state_completion_df['COUNT'] / state_completion_df.groupby(['STFIPS'])['COUNT'].transform('sum')

state_completion_df = state_completion_df.replace({'STFIPS': STATE_DICT})
state_completion_df = state_completion_df.replace({'REASON': REASON_DICT})

In [ ]:
%matplotlib inline

import altair as alt

#completion_df.groupby(['STFIPS'])['COUNT'].plot.barh(stacked=True,title='DISCHARGE RATES BY STATE', figsize=(8,10))

# bar = alt.Chart(state_completion_df).mark_bar().encode(
#     y='STFIPS:N',
#     x='PERC:Q',
#     color='REASON:N',
# #    order=alt.Order('PERC', sort='ascending'
# #                   )
# )

bar = alt.Chart(state_completion_df).mark_bar().transform_calculate(
        filtered='datum.REASON == "1 - Treatment Completed" ? datum.PERC : 0'
        ).encode(
            x=alt.X('PERC:Q', scale=alt.Scale(domain=(0,100)), axis=alt.Axis(title='Percentage of Cases')),
            y=alt.Y('STFIPS:N', sort=alt.SortField('filtered', order = 'descending'), axis=alt.Axis(title='State')),
            color='REASON:N',
            order=alt.Order('REASON',sort='ascending'))

bar

### Choropleth Testing

In [ ]:
state_reason1_df = state_completion_df[state_completion_df['REASON']=='1 - Treatment Completed']

In [ ]:
state_abbv_df = state_completion_df.replace({'STFIPS':US_STATE_TO_ABBREV})

state_abbv_df

In [ ]:
map_chart(state_abbv_df, 
          'PERC', 
          1, 
          'Completion Percentage', 
          '2019 Substance Treatment Completion Rates by State',
          states='ALL',
          is_static=True)

### Region/Census Division

In [ ]:
df2 = df.copy()
completion_df = df2.groupby(['DIVISION','REASON']).size().reset_index()
completion_df = completion_df.rename(columns={0:'COUNT'})
completion_df['PERC'] = 100 * completion_df['COUNT'] / completion_df.groupby(['DIVISION'])['COUNT'].transform('sum')

bar = alt.Chart(completion_df).mark_bar().encode(
    x='DIVISION:N',
    y='PERC',
    color='REASON:N',
#    order=alt.Order('PERC', sort='ascending'
#                   )
)

bar

### Treatment Type @ Admission

In [ ]:
df3 = df.copy()
completion_df = df3.groupby(['SERVICES','REASON']).size().reset_index()
completion_df = completion_df.rename(columns={0:'COUNT'})
completion_df['PERC'] = 100 * completion_df['COUNT'] / completion_df.groupby(['SERVICES'])['COUNT'].transform('sum')

bar = alt.Chart(completion_df).mark_bar().encode(
    x='SERVICES:N',
    y='PERC',
    color='REASON:N',
#    order=alt.Order('PERC', sort='ascending'
#                   )
)

bar

### Treatment Type @ Discharge

In [ ]:
df4 = df.copy()
completion_df = df4.groupby(['SERVICES_D','REASON']).size().reset_index()
completion_df = completion_df.rename(columns={0:'COUNT'})
completion_df['PERC'] = 100 * completion_df['COUNT'] / completion_df.groupby(['SERVICES_D'])['COUNT'].transform('sum')

bar = alt.Chart(completion_df).mark_bar().encode(
    x='SERVICES_D:N',
    y='PERC',
    color='REASON:N',
#    order=alt.Order('PERC', sort='ascending'
#                   )
)

bar